# wrds_utils

> Common utilities needed to download and process data from WRDS

In [ ]:
#| default_exp wrds_utils

In [ ]:
#|exports
from __future__ import annotations
from typing import List, Dict, Tuple, Callable
import os

import pandas as pd
import numpy as np
import wrds 

In [ ]:
#|export
def download(sql_string: str=None,
             wrds_username: str=None, #If None, looks for WRDS_USERNAME with `os.getenv`, then prompts you if needed
             ) -> pd.DataFrame:
    """Downloads data from WRDS using the given `sql_string`"""

    if wrds_username is None:
        wrds_username = os.getenv('WRDS_USERNAME')
        if wrds_username is None: wrds_username = input("Enter your WRDS username: ") 

    dbconn = wrds.Connection(wrds_username = wrds_username)
    try : df = dbconn.raw_sql(sql=sql_string)
    finally: dbconn.close() 
    return df

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()